In [ ]:
'''
Licensed Materials - Property of IBM
IBM Maximo APM - Predictive Maintenance Insights On-Premises
IBM Maximo APM - Predictive Maintenance Insights SaaS 
IBM Maximo Application Suite 
© Copyright IBM Corp. 2019, 2020, 2021 All Rights Reserved.
US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with IBM Corp.
'''

# Maximo Predict - Anomaly Detection Model Template

1. [Introduction](#introduction)
2. [Install Maximo Predict SDK](#install-maximo-apm-pmi-sdk)
3. [Setup the Model Training Pipeline](#setup-model-training-pipline)
4. [Train the Model Instance](#train-model-instance)
5. [Register the Trained Model Instance](#register-trained-model-instance)
6. [Model Template Internals](#model-template-internals)

<a id='introduction'></a>
## Introduction

IoT sensors are pervasive in the manufacturing industries for monitoring assets. The data these sensors capture can be combined with advanced statistical and machine learning algorithms to yield insight on pending failures and optimal maintenance and operation strategies. A gas turbine, compressor, and boat engine are examples of valuable and costly assets that can benefit from such insights. These industrial assets fail due to many reasons, and often they behave in abnormal ways preceding the behaviour. In order to have an effective predictive maintenance program, it is crucial to be able to detect these abnormalities so that action can be taken before costly downtimes occur due to unexpected failures. However, identifying these tell-tale abnormalities before a failure event is difficult since, in general, failures are comparatively rare events so there is often a scarcity of tagged data for building predictive models. In the absence of abundant tagged (supervised) failure data, anomaly detection techniques as described in this notebook provide an unsupervised way of learning a model from data that mostly contains “normal” operating data with a sparse number of actual failure events.

Maximo Predict automated anomaly learning framework (see the following figure) explores various anomaly detection techniques to identify an optimal one. The anomaly framework is divided into an anomaly trainer and anomaly evaluator: The anomaly model is first learned on the training data that is assumed to be “normal” by anomaly trainer. In the following example figure, we plot two failures observed for an asset over a time. The normal data are selected from a time window that is far from the time were the failure occurred (i.e., the asset is operating normally). The trained model is applied on the validation data (mostly data coming from the pre-failure window in case of time series data) to obtain the anomaly validator. In anomaly validator, we calculate the performance of each anomaly model with respect to the anomaly label, where samples very near the failures are assigned anomaly label 1 (see 0 and 1 in pre-failure window). In summary, a user must provide two datasets for anomaly learning:

1. Normal Data to train the anomaly model. All samples are from one class only (i.e., class 0)
2. Validation Data to do anomaly scoring and evaluation. Samples are from pre-failure windows with samples close to failures are labelled as a class 

<a id="install-maximo-apm-pmi-sdk"></a>
## Install Maximo Predict SDK

You will need follow 3 credentials to run the notebook. You can obtain them if you are admin on Maximo Predict UI.
 + **APM_ID**: Application Administration -> System Properties -> Filter -> Search PMIId -> Current Value (eg. b95ed774)
 + **APM_API_BASEURL**: Application Administration -> Integration -> End Points--> Searh for predict -> click search result PREDICTAPI -> URL (e.g https://predict-api.mas-pmidev1-predict.svc, note you just need first part of the url)
 + **APM_API_KEY**: Application Administration -> Go To Administration ->Integration-> API Keys. Copy key from user card (e.g. 6805t46gn3tef37pu0picpg9vcq3hsmamm1enc), or Add API key if not exist.

In [ ]:
!pip uninstall -y pmlib
!pip uninstall -y srom
!pip uninstall -y dqlearn

In [ ]:
import json
from project_lib import Project
project = Project.access()
store = json.load(project.get_file("fast_execution.json"))
store
notebook_name = "PMI - Anomaly Detection -UnSupervised"
notebook_precursors = ["FastStartDataLoader_Predict"]
store[notebook_name] = {"precursors":notebook_precursors, "outputs":{}}
store_json = json.dumps(store, indent =4)
project.save_data("AD_execution.json", store_json, overwrite=True)
print("Notebook added to json")
for precursor in notebook_precursors:
    for nb in store:
        if precursor == nb:
            #Found precursor. Check if it has output(meaning it has been run)
            if store[precursor]["outputs"] == {}:
                #Precursor has not been run. Must run before notebook can be run
                raise ValueError(precursor + " must run before this notebook.")

In [ ]:
import json
import os
# Opening JSON file
f = open('/project_data/data_asset/Predict_Envs.json',)
data = json.load(f)
print(data)
f.close()
os.environ['APM_ID'] = data['APM_ID']
os.environ['APM_API_BASEURL'] = data['APM_API_BASEURL']
os.environ['APM_API_KEY'] = data['APM_API_KEY']


In [ ]:
import os
os.environ['TRUST_PREDICT']= os.getenv('APM_API_BASEURL')[8:]
print(os.getenv('TRUST_PREDICT'))

In [ ]:
!pip install --trusted-host ${TRUST_PREDICT} -U ${APM_API_BASEURL}/ibm/pmi/service/rest/ds/${APM_ID}/${APM_API_KEY}/lib/download?filename=pmlib


Then, install Predict SDK with `pip`.

In [ ]:
import pkg_resources

dists = [d for d in pkg_resources.working_set]

if not('pyspark' in dists):
    print ("pyspark does not exist. Installing ..." )
    !pip install pyspark==3.0.1 
else:
    print ("pyspark already exists. Not installing")

Import pmlib

In [ ]:
import srom
srom.__version__

In [ ]:
import pmlib
from pmlib import api
#from pmlib.api import init_environ
#from pmlib.util import current_directory
#from iotfunctions.db import Database
#pmlib.setup_logging()
api.init_environ()

In [ ]:
#use the following flags to suppress the pointless pyspark logs that fill up space
import logging
logging.getLogger('py4j.java_gateway').setLevel(logging.WARN)

<a id="setup-model-training-pipline"></a>
## Setup the Model Training Pipeline

Before you can start working on the model training pipeline, you have to setup an asset group and asset-sensor relationshp properly in Maximo. See IBM Maximo APM - Predictive Maintenance Insights SaaS User Guide for details.

Required model pipeline configuration:

* Asset group ID: The unit of model processing is an asset group. Asset groups are managed on Maximo Health->Predict grouping UI. You need to get the ID of the asset group to be analyzed by this model.
* Sensor data as features: This model also accepts one or more features from either asset data or IOT data. **Note that these features must be of numeric type.** Features are specifed simply by attribute names prefixed by type (with separator colon). For asset attribute, the prefix is an empty string. For IOT data attribute, the prefix is the device type registered on Watson IOT Platform.
* Prediction output names: This model generates two outputs, the calculated anomaly score and the anomaly threshold. Give them names (containing only alphanumeric, dash, and underline). **Note that the order of the names matters, the first must be the calculated anomaly score.** Additionally a third value is generated with a 0 (No anomaly detected) or 1 (Anomaly detected) derived from the first two outputs.
* SROM training options: This model requires much higher computation compared to the other out of box models, hence it usually takes very long to complete the training on Watson Studio. It might be necessary to use Python with Spark environment (with 10 executors) on Watson Studio to increase the training speed. You can specify either **``"exectype": "spark_node_random_search"```** or **```"exectype": "spark_node_complete_search"```** to enable this.


Now you can setup a training pipeine based on this model template, with your own data, to train a model instance.

Assume you have set up a device type in IOT Platform.

You can find the asset_group_id in the Maximo Health -> Predict grouping page.

In [ ]:
print(store)

asset_group_id=store["FastStartLoader-Predict"]["outputs"]["asset_group_id"] 

print(asset_group_id)

device_type_qualified_features will used as features in the AnomalyDetectionAssetGroupPipeline

In [ ]:
#  Assume that you have set up device_type 'Pump_Afm' in Monitor. Assume that 'Pump_Afm' has the 'VELOCITYX','VELOCITYY','VELOCITYZ'.
device_type=store["FastStartLoader-Predict"]["outputs"]["device_type"]
iot_type=device_type
desired_features = ['VELOCITYX','VELOCITYY','VELOCITYZ']
device_type_qualified_features = [device_type+':'+col for col in desired_features]
print(device_type_qualified_features)




scoring_strategy can have one of three values:
scoring_strategy='al_score' or scoring_strategy='em_score'  or scoring_strategy='mv_score' 

In [ ]:
scoring_strategy='al_score'  # scoring_strategy can be em_score or mv_score

In [ ]:
import json
import os
import numpy as np
import pandas as pd
#import srom.utils.tabulate as tabulate

import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import HTML, display, Image

from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture
from sklearn.svm import OneClassSVM
from sklearn.covariance import (EmpiricalCovariance, EllipticEnvelope, LedoitWolf, MinCovDet, OAS, ShrunkCovariance)
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, MaxAbsScaler, Normalizer, QuantileTransformer, PowerTransformer, PolynomialFeatures

import srom
from srom.pipeline.anomaly_pipeline import AnomalyPipeline
from srom.pipeline.srom_param_grid import SROMParamGrid
from srom.anomaly_detection.algorithms.anomaly_ensembler import AnomalyEnsembler
from srom.anomaly_detection.generalized_anomaly_model import GeneralizedAnomalyModel
from srom.anomaly_detection.gaussian_graphical_anomaly_model import GaussianGraphicalModel
from srom.anomaly_detection.algorithms import NearestNeighborAnomalyModel, LOFNearestNeighborAnomalyModel
from srom.anomaly_detection.algorithms.pca_t2 import AnomalyPCA_T2
from srom.anomaly_detection.algorithms.pca_q import AnomalyPCA_Q
from srom.anomaly_detection.algorithms.bayesian_gmm_outlier import BayesianGMMOutlier
from srom.anomaly_detection.algorithms.gmm_outlier import GMMOutlier
from srom.anomaly_detection.algorithms.anomaly_robust_pca import AnomalyRobustPCA
from srom.anomaly_detection.algorithms.neural_network_nsa import NeuralNetworkNSA
from srom.anomaly_detection.algorithms.ggm_quic import GraphQUIC
from srom.anomaly_detection.algorithms.ggm_pgscps import GraphPgscps
from srom.anomaly_detection.algorithms.sample_svdd import SampleSVDD
from srom.anomaly_detection.algorithms.random_partition_forest import RandomPartitionForest
from srom.anomaly_detection.algorithms.extended_isolation_forest import ExtendedIsolationForest
from srom.anomaly_detection.algorithms.negative_sample_anomaly import NSA
from srom.utils.no_op import NoOp
from srom.utils import file_utils, s3utils
import dqlearn

sns.set()
sns.set_style("whitegrid")
%matplotlib inline


#### There are three modes in which the Unsupervised Learning pipeline can be executed
+ **default** - this executes a pipeline with multiple scalers and transformations using selected set of classifiers / estimators. This set is smaller than the one for the semi-supervised model pipeline
+ **qfunction** - this computes the anomaly threshold based on qfunction. This is specified using the variable `threshold_computation_strategy`. Specifying this requires another parameter `qfunction_threshold`. If this parameter is not provided the default value of 0.9 will be used. Lower the value of `qfunction_threshold` if more records need to be identified as anomalies. Raise this value if the number of anomalies identified is high including misclassifications and therefore needs to  be reduced
+ **meanabsolutedev** - this computes the anomaly threshold based on mean absolute deviation. This is specified using the variable `threshold_computation_strategy`. Specifying this requires another parameter `meanabsolutedev_threshold`. If this parameter is not provided the default value of 2.5 will be used. Raising this value could reduce the number of false positives and identifications of anomaly. Fine tune this as needed
+ **contamination** - this computes the anomaly threshold based on expected contamination. This is specified using the variable `threshold_computation_strategy`. Specifying this requires another parameter `contamination`. If this parameter is not provided the default value of 0.1 will be used. In addition, this approach also requires the user to specifiy the exact classifier to be tried from the list of enumerated types in `UnsupervisedADLearningAlgorithm`. See the following example for illustration. This does not try a large pipeline like the previous approaches, but instead it takes the user input for the scaler and the specific classifier to fit. # Raise this value if more records need to be identified as anomalies. Lower this value if the number of anomalies identified is high including misclassifications, and therefore needs to  be reduced

**The configuration for the Asset Group Pipeline for each mode is described below**

The examples below configure a pipeline for this model, accepting **```var_1```through ```var_8```** of Watson IOT Platform device type **```LLL_ROBOT_TYPE```**. The predicted output of the trained model instance is called **```anomaly_score```** (for calculated anomaly score) , **```anomaly_threshold```** (for anomaly threshold)  and **```anomaly_detected```** (1 means Anomaly is detected. 0 means Anomaly is not detected.).

In [ ]:
device_type_qualified_features

In [ ]:
model_pipeline={
                'features':device_type_qualified_features,
                'features_resampled': {
                        iot_type: {
                            '${freqency}': '1D',
                            'VELOCITYX': {
                                'mean': None,
                            },
                            'VELOCITYY': {
                                'mean': None,
                            },
                            'VELOCITYZ': {
                                'mean': None,
                            },
                        },
                    },              
                'predictions': ['anomaly_score', 'anomaly_threshold', 'anomaly_detected'],               
                'data_quality_advisor': True,               
                "data_quality_sensor_threshold":0.6,
                #'threshold_computation_strategy':'default',
                'scoring_strategy': scoring_strategy,
                'use_labeled_data': False,  # For un-supervised anomaly detection, set it to be False.              
}
print(model_pipeline)

In [ ]:
print(asset_group_id)

In [ ]:
group = pmlib.AnomalyDetectionAssetGroupPipeline(
            asset_group_id=asset_group_id,
            model_pipeline=model_pipeline)

<a id="train-model-instance"></a>
## Train the Model Instance

Now you can train the model instance. 

In [ ]:
%time df=group.execute()

<a id="register-trained-model-instance"></a>
## Register the Trained Model Instance

In [ ]:
group.register()

<a id="eanble-trained-model-instance"></a>
## Enable the Trained Model Instance and set up schedule in Monitor

In [ ]:
# All time is UTC time, see following example to set schedule to run every 5 minute, every hour, every day, every week, every month
# minute and second will be ignored. "2021-04-12 15:12:15" is same as "2021-04-12 15:00:00" in following examples
#group.enable(enabled=True, schedule={"starting_at": "2021-04-09 15:35:15", "every": "5min"})
#group.enable(enabled=True, schedule={"starting_at": "2021-04-12 15:12:15", "every": "1H"})
#group.enable(enabled=True, schedule={"starting_at": "2021-04-12 15:12:15", "every": "1D"})
#group.enable(enabled=True, schedule={"starting_at": "2021-04-12 15:12:15", "every": "1W"})
#group.enable(enabled=True, schedule={"starting_at": "2021-04-12 15:12:15", "every": "1M"})
group.enable(enabled=True,schedule={"starting_at": "05:00:01", "every": "5min"})

In [ ]:
from datetime import datetime
import dateutil.relativedelta
# datetime object containing current date and time

now = datetime.now() 
dt_string = now.strftime("%Y-%m-%d")
print("date and time =", dt_string)


now = datetime.now()
start = now + dateutil.relativedelta.relativedelta(months=-1)
print(start.strftime("%Y-%m-%d"))


df=group.execute(start_ts=start, end_ts=now)


In [ ]:
group._write(df=df)

In [ ]:
raise Exception("exit")

# How to configure context anomaly 

1. You need to provide the context name in the model_pipeline. 
2. You need to provide the context_expression in the model_pipeline. Please see the below example.

In [ ]:
context_model_pipeline={
                'features':device_type_qualified_features,
                'features_resampled': {
                        iot_type: {
                            '${freqency}': '1H',
                            'VELOCITYX': {
                                'mean': None,
                            },
                            'VELOCITYY': {
                                'mean': None,
                            },
                            'VELOCITYZ': {
                                'mean': None,
                            },
                        },
                    },
                
                "aggregation_methods": ["max", "min"],  #for feature engineering
                "rolling_window_size": "2H",  #for feature engineering
                
                
                
                'context_expression' : 'VELOCITYX > 0.00006 & VELOCITYZ > 0.000226',
                'context_name': 'your_brand_new_context_name',
                
                
                'predictions': ['anomaly_score', 'anomaly_threshold', 'anomaly_detected'],
                
                'data_quality_advisor': True,
                
                "data_quality_sensor_threshold":0.6,
                #'threshold_computation_strategy':'default',
                'scoring_strategy': scoring_strategy,
                'use_labeled_data': False,  # For un-supervised anomaly detection, set it to be False.
                
                
}

print(model_pipeline)

In [ ]:


group = pmlib.AnomalyDetectionAssetGroupPipeline(
            asset_group_id=asset_group_id,
            model_pipeline=context_model_pipeline)

In [ ]:
# %time df=group.execute()

In [ ]:
df

In [ ]:
# group.register()

In [ ]:
# All time is UTC time, see following example to set schedule to run every 5 minute, every hour, every day, every week, every month
# minute and second will be ignored. "2021-04-12 15:12:15" is same as "2021-04-12 15:00:00" in following examples
#group.enable(enabled=True, schedule={"starting_at": "2021-04-09 15:35:15", "every": "5min"})
#group.enable(enabled=True, schedule={"starting_at": "2021-04-12 15:12:15", "every": "1H"})
#group.enable(enabled=True, schedule={"starting_at": "2021-04-12 15:12:15", "every": "1D"})
#group.enable(enabled=True, schedule={"starting_at": "2021-04-12 15:12:15", "every": "1W"})
#group.enable(enabled=True, schedule={"starting_at": "2021-04-12 15:12:15", "every": "1M"})
# group.enable(enabled=True,schedule={"starting_at": "05:00:01", "every": "5min"})

# How to override training stage 

You can use override_training_stages to override training stage.

In [ ]:

model_pipeline={
                'features':device_type_qualified_features,
                'features_resampled': {
                        iot_type: {
                            '${freqency}': '1H',
                            'VELOCITYX': {
                                'mean': None,
                            },
                            'VELOCITYY': {
                                'mean': None,
                            },
                            'VELOCITYZ': {
                                'mean': None,
                            },
                        },
                    },
                
               
                
                
                'predictions': ['anomaly_score', 'anomaly_threshold', 'anomaly_detected'],
                
                'data_quality_advisor': True,
                
                "data_quality_sensor_threshold":0.6,
                #'threshold_computation_strategy':'default',
                'scoring_strategy': scoring_strategy,
                'use_labeled_data': False,
                
                'override_training_stages': [
                    
                    [
                        ('isolationforest',GeneralizedAnomalyModel(base_learner=IsolationForest(), fit_function='fit', predict_function='decision_function', score_sign=-1)),
                        #('graphpgscps',GeneralizedAnomalyModel(base_learner=GraphPgscps(), fit_function='fit', predict_function='mahalanobis', score_sign=1))
                        
                    ]
                ]
}

print(model_pipeline)
                

In [ ]:
from sklearn.ensemble import IsolationForest
from srom.anomaly_detection.algorithms import GraphPgscps
from sklearn.svm import OneClassSVM

group = pmlib.AnomalyDetectionAssetGroupPipeline(
            asset_group_id=asset_group_id,
            model_pipeline=model_pipeline)

# How to Re-train the model 

#### find the model_instance_id from the Health->Predict grouping

In [ ]:
#group_ad_qfunction = pmlib.AnomalyDetectionAssetGroupPipeline(
#            asset_group_id=asset_group_id, model_instance_id='62E2DC56-F3FB-4EA5-AB3E-93E08FD616C2')
#group_ad_qfunction.unregister('62E2DC56-F3FB-4EA5-AB3E-93E08FD616C2', force=True)

In [ ]:
group_ad = pmlib.AnomalyDetectionAssetGroupPipeline(
            asset_group_id=asset_group_id,
            model_instance_id='50D47076-99B5-4AD0-ADEB-820ABFE888DD',
            model_pipeline={
                'features':device_type_qualified_features,
                
                
                'predictions': ['anomaly_score', 'anomaly_threshold', 'anomaly_detected'],
                
                "data_quality_sensor_threshold":0.6,
                'threshold_computation_strategy':'default',
                'use_labeled_data': False,
                
               
            })

In [ ]:
group_ad.unregister('50D47076-99B5-4AD0-ADEB-820ABFE888DD',force=True)

##### Then you can re-train the model now.

In [ ]:
from sklearn.ensemble import IsolationForest
from srom.anomaly_detection.algorithms import GraphPgscps
from sklearn.svm import OneClassSVM

group = pmlib.AnomalyDetectionAssetGroupPipeline(
            asset_group_id=asset_group_id,
            model_pipeline={
                'features':device_type_qualified_features,
                'features_resampled': {
                        iot_type: {
                            '${freqency}': '1H',
                            'VELOCITYX': {
                                'mean': None,
                            },
                            'VELOCITYY': {
                                'mean': None,
                            },
                            'VELOCITYZ': {
                                'mean': None,
                            },
                        },
                    },
                
                "aggregation_methods": ["max", "min"],
                "rolling_window_size": "2H",               
                
                'context_expression' : 'VELOCITYX > 0.0006 & VELOCITYZ > 0.000226',
                'context_name': 'my_context_name',
                               
                'predictions': ['anomaly_score', 'anomaly_threshold', 'anomaly_detected'],
                
                'data_quality_advisor': False,
                
                "data_quality_sensor_threshold":0.6,
                'threshold_computation_strategy':'default',
                'scoring_strategy':'mv_score',
                'use_labeled_data': False,
                
                'override_training_stages': [
                    
                    [
                        ('isolationforest',GeneralizedAnomalyModel(base_learner=IsolationForest(), fit_function='fit', predict_function='decision_function', score_sign=-1)),
                      #  ('graphpgscps',GeneralizedAnomalyModel(base_learner=GraphPgscps(), fit_function='fit', predict_function='mahalanobis', score_sign=1))
                        
                    ]
                ]
                
               
            })

In [ ]:
# %time df=group.execute()

In [ ]:
store_json = json.dumps(store, indent = 2)
project.save_data("AD_execution.json", store_json, overwrite=True)